In [8]:
from google.colab import drive
import os
drive.mount('/content/drive/',force_remount=True)
os.chdir("/content/drive/MyDrive/Colab Notebooks/LearningToCountEverything")
os.listdir()

Mounted at /content/drive/


['test.py',
 'README.md',
 'utils.py',
 'model.py',
 'logsSave',
 '__pycache__',
 'data',
 '.git',
 'train.py',
 'AML-LTCE-Train.ipynb',
 'AML-LTCE-Test.ipynb',
 'utils_DO_NOT_TOUCH.ipynb']

In [9]:
import torch.nn as nn
from model import  Resnet50FPN,CountRegressor,weights_normal_init
from utils import MAPS, Scales, Transform,TransformTrain,extract_features, visualize_output_and_save
from PIL import Image
import os
import torch
import argparse
import json
import numpy as np
from tqdm import tqdm
from os.path import exists,join
import random
import torch.optim as optim
import torch.nn.functional as F

In [10]:
"""parser = argparse.ArgumentParser(description="Few Shot Counting Evaluation code")
parser.add_argument("-dp", "--data_path", type=str, default='/content/drive/MyDrive/Colab Notebooks/LearningToCountEverything/data/', help="Path to the FSC147 dataset")
parser.add_argument("-o", "--output_dir", type=str,default="./logsSave", help="/Path/to/output/logs/")
parser.add_argument("-ts", "--test-split", type=str, default='val', choices=["train", "test", "val"], help="what data split to evaluate on on")
parser.add_argument("-ep", "--epochs", type=int,default=1500, help="number of training epochs")
parser.add_argument("-g", "--gpu", type=int,default=0, help="GPU id")
parser.add_argument("-lr", "--learning-rate", type=float,default=1e-5, help="learning rate")
#args = parser.parse_args()
args = parser.parse_args(argv[1:])"""

'parser = argparse.ArgumentParser(description="Few Shot Counting Evaluation code")\nparser.add_argument("-dp", "--data_path", type=str, default=\'/content/drive/MyDrive/Colab Notebooks/LearningToCountEverything/data/\', help="Path to the FSC147 dataset")\nparser.add_argument("-o", "--output_dir", type=str,default="./logsSave", help="/Path/to/output/logs/")\nparser.add_argument("-ts", "--test-split", type=str, default=\'val\', choices=["train", "test", "val"], help="what data split to evaluate on on")\nparser.add_argument("-ep", "--epochs", type=int,default=1500, help="number of training epochs")\nparser.add_argument("-g", "--gpu", type=int,default=0, help="GPU id")\nparser.add_argument("-lr", "--learning-rate", type=float,default=1e-5, help="learning rate")\n#args = parser.parse_args()\nargs = parser.parse_args(argv[1:])'

In [11]:
data_path = '/content/drive/MyDrive/Colab Notebooks/LearningToCountEverything/data/'
output_dir = "./logsSave"
test_split = "val" #choices=["train", "test", "val"]
epochs = 1500
gpu = 0 
learning_rate = 1e-5

In [12]:
#data_path = args.data_path
anno_file = data_path + 'annotation_FSC147_384.json'
data_split_file = data_path + 'Train_Test_Val_FSC_147.json'
im_dir = data_path + 'images_384_VarV2'
gt_dir = data_path + 'gt_density_map_adaptive_384_VarV2'

if not exists(output_dir):
    os.mkdir(output_dir)

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu)

criterion = nn.MSELoss().cuda()

resnet50_conv = Resnet50FPN()
resnet50_conv.cuda()
resnet50_conv.eval()

regressor = CountRegressor(6, pool='mean')
weights_normal_init(regressor, dev=0.001)
regressor.train()
regressor.cuda()
optimizer = optim.Adam(regressor.parameters(), lr = learning_rate)

with open(anno_file) as f:
    annotations = json.load(f)

with open(data_split_file) as f:
    data_split = json.load(f)

In [13]:
def train():
    print("Training on FSC147 train set data")
    im_ids = data_split['train']
    random.shuffle(im_ids)
    train_mae = 0
    train_rmse = 0
    train_loss = 0
    pbar = tqdm(im_ids)
    cnt = 0
    for im_id in pbar:
        cnt += 1
        anno = annotations[im_id]
        bboxes = anno['box_examples_coordinates']
        dots = np.array(anno['points'])

        rects = list()
        for bbox in bboxes:
            x1 = bbox[0][0]
            y1 = bbox[0][1]
            x2 = bbox[2][0]
            y2 = bbox[2][1]
            rects.append([y1, x1, y2, x2])

        image = Image.open('{}/{}'.format(im_dir, im_id))
        image.load()
        density_path = gt_dir + '/' + im_id.split(".jpg")[0] + ".npy"
        density = np.load(density_path).astype('float32')    
        sample = {'image':image,'lines_boxes':rects,'gt_density':density}
        sample = TransformTrain(sample)
        image, boxes,gt_density = sample['image'].cuda(), sample['boxes'].cuda(),sample['gt_density'].cuda()

        with torch.no_grad():
            features = extract_features(resnet50_conv, image.unsqueeze(0), boxes.unsqueeze(0), MAPS, Scales)
        features.requires_grad = True
        optimizer.zero_grad()
        output = regressor(features)

        #if image size isn't divisible by 8, gt size is slightly different from output size
        if output.shape[2] != gt_density.shape[2] or output.shape[3] != gt_density.shape[3]:
            orig_count = gt_density.sum().detach().item()
            gt_density = F.interpolate(gt_density, size=(output.shape[2],output.shape[3]),mode='bilinear')
            new_count = gt_density.sum().detach().item()
            if new_count > 0: gt_density = gt_density * (orig_count / new_count)
        loss = criterion(output, gt_density)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        pred_cnt = torch.sum(output).item()
        gt_cnt = torch.sum(gt_density).item()
        cnt_err = abs(pred_cnt - gt_cnt)
        train_mae += cnt_err
        train_rmse += cnt_err ** 2
        pbar.set_description('actual-predicted: {:6.1f}, {:6.1f}, error: {:6.1f}. Current MAE: {:5.2f}, RMSE: {:5.2f} Best VAL MAE: {:5.2f}, RMSE: {:5.2f}'.format( gt_cnt, pred_cnt, abs(pred_cnt - gt_cnt), train_mae/cnt, (train_rmse/cnt)**0.5,best_mae,best_rmse))
        print("")
    train_loss = train_loss / len(im_ids)
    train_mae = (train_mae / len(im_ids))
    train_rmse = (train_rmse / len(im_ids))**0.5
    return train_loss,train_mae,train_rmse

In [16]:
def eval():
    cnt = 0
    SAE = 0 # sum of absolute errors
    SSE = 0 # sum of square errors

    print("Evaluation on {} data".format(test_split))
    im_ids = data_split[test_split]
    pbar = tqdm(im_ids)
    for im_id in pbar:
        anno = annotations[im_id]
        bboxes = anno['box_examples_coordinates']
        dots = np.array(anno['points'])

        rects = list()
        for bbox in bboxes:
            x1 = bbox[0][0]
            y1 = bbox[0][1]
            x2 = bbox[2][0]
            y2 = bbox[2][1]
            rects.append([y1, x1, y2, x2])

        image = Image.open('{}/{}'.format(im_dir, im_id))
        image.load()
        sample = {'image':image,'lines_boxes':rects}
        sample = Transform(sample)
        image, boxes = sample['image'].cuda(), sample['boxes'].cuda()

        with torch.no_grad():
            output = regressor(extract_features(resnet50_conv, image.unsqueeze(0), boxes.unsqueeze(0), MAPS, Scales))

        gt_cnt = dots.shape[0]
        pred_cnt = output.sum().item()
        cnt = cnt + 1
        err = abs(gt_cnt - pred_cnt)
        SAE += err
        SSE += err**2

        pbar.set_description('{:<8}: actual-predicted: {:6d}, {:6.1f}, error: {:6.1f}. Current MAE: {:5.2f}, RMSE: {:5.2f}'.format(im_id, gt_cnt, pred_cnt, abs(pred_cnt - gt_cnt), SAE/cnt, (SSE/cnt)**0.5))
        print("")

    print('On {} data, MAE: {:6.2f}, RMSE: {:6.2f}'.format(args.test_split, SAE/cnt, (SSE/cnt)**0.5))
    return SAE/cnt, (SSE/cnt)**0.5

In [7]:
best_mae, best_rmse = 1e7, 1e7
stats = list()
for epoch in range(0,epochs):
    regressor.train()
    train_loss,train_mae,train_rmse = train()
    regressor.eval()
    val_mae,val_rmse = eval()
    stats.append((train_loss, train_mae, train_rmse, val_mae, val_rmse))
    stats_file = join(args.output_dir, "stats" +  ".txt")
    with open(stats_file, 'w') as f:
        for s in stats:
            f.write("%s\n" % ','.join([str(x) for x in s]))    
    if best_mae >= val_mae:
        best_mae = val_mae
        best_rmse = val_rmse
        model_name = args.output_dir + '/' + "FamNet.pth"
        torch.save(regressor.state_dict(), model_name)

    print("Epoch {}, Avg. Epoch Loss: {} Train MAE: {} Train RMSE: {} Val MAE: {} Val RMSE: {} Best Val MAE: {} Best Val RMSE: {} ".format(
              epoch+1,  stats[-1][0], stats[-1][1], stats[-1][2], stats[-1][3], stats[-1][4], best_mae, best_rmse))

Training on FSC147 train set data


  0%|          | 0/3659 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3635: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)
actual-predicted:   67.0,    0.0, error:   67.0. Current MAE: 67.00, RMSE: 67.00 Best VAL MAE: 10000000.00, RMSE: 10000000.00:   0%|          | 1/3659 [01:07<68:55:02, 67.82s/it]

actual-predicted:   24.0,    2.3, error:   21.7. Current MAE: 44.37, RMSE: 49.81 Best VAL MAE: 10000000.00, RMSE: 10000000.00:   0%|          | 2/3659 [01:08<28:58:43, 28.53s/it]

actual-predicted:  119.0,    4.0, error:  115.0. Current MAE: 67.93, RMSE: 77.88 Best VAL MAE: 10000000.00, RMSE: 10000000.00:   0%|          | 3/3659 [01:09<16:09:28, 15.91s/it]

actual-predicted:  106.0,    5.3, error:  100.7. Current MAE: 76.11, RMSE: 84.16 Best VAL MAE: 10000000.00, RMSE: 10000000.00:   0%|          | 4/3659 [01:10<10:04:51,  9.93s/it]

...

actual-predicted:   40.0,   53.9, error:   13.9. Current MAE: 45.49, RMSE: 74.41 Best VAL MAE: 10000000.00, RMSE: 10000000.00:   6%|▌         | 213/3659 [04:25<51:58,  1.11it/s]

actual-predicted:   12.0,   60.7, error:   48.7. Current MAE: 45.50, RMSE: 74.31 Best VAL MAE: 10000000.00, RMSE: 10000000.00:   6%|▌         | 214/3659 [04:26<53:47,  1.07it/s]

actual-predicted:   12.0,   63.6, error:   51.6. Current MAE: 45.53, RMSE: 74.22 Best VAL MAE: 10000000.00, RMSE: 10000000.00:   6%|▌         | 215/3659 [04:27<56:48,  1.01it/s]

actual-predicted:   12.0,   63.6, error:   51.6. Current MAE: 45.53, RMSE: 74.22 Best VAL MAE: 10000000.00, RMSE: 10000000.00:   6%|▌         | 215/3659 [04:28<1:11:33,  1.25s/it]


KeyboardInterrupt: ignored